<a href="https://colab.research.google.com/github/Arun-nexus/nexux/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d denvermagtibay/ai-generated-essays-dataset

In [ ]:
import zipfile
with zipfile.ZipFile("ai-generated-essays-dataset.zip","r") as zip:
  zip.extractall("dataset")

In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv("/content/dataset/AI Generated Essays Dataset.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df["generated"].value_counts()

In [ ]:
from transformers import DistilBertTokenizer,TFDistilBertForSequenceClassification

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
text_train,text_val,labels_train,labels_val=train_test_split(df["text"].to_list(),df["generated"],test_size=0.2,random_state=7)

In [ ]:
text_train=tokenizer(text_train,truncation=True,padding=True,max_length=500)
text_val=tokenizer(text_val,truncation=True,padding=True,max_length=500)


In [ ]:
import tensorflow as tf

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((
        {
        "input_ids": tf.convert_to_tensor(text_train['input_ids']),
        "attention_mask": tf.convert_to_tensor(text_train['attention_mask'])
    },
    tf.convert_to_tensor(labels_train))).shuffle(1000).batch(3)

In [ ]:
val_dataset=tf.data.Dataset.from_tensor_slices((
        {
        "input_ids": tf.convert_to_tensor(text_val['input_ids']),
        "attention_mask": tf.convert_to_tensor(text_val['attention_mask'])
    },
    tf.convert_to_tensor(labels_val))).batch(3)

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
model=TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),optimizer="Adam",metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
stop=EarlyStopping(
    monitor="val_sparse_categotical_accuracy",
    patience=2,
    verbose=1,
    restore_best_weights=True
)

In [ ]:
hist=model.fit(train_dataset,validation_data=val_dataset,epochs=10,batch_size=512)

In [ ]:
model.evaluate(val_dataset)

In [ ]:
def predict(text):
  inputs=tokenizer(text,return_tensor="tf",truncation=True,padding=True,max_length=500)
  logits=model(inputs).logits
  prob=tf.nn.softmax(logits,axis=-1)[0].numpy()
  return {"me":float(prob[0]),"model":float(prob[1])}

predict("this essay was written")